# Fisheries Training Part 1 - Harvest Optimization and MOEA Diagnostics

This is the second post in a training series, studying decision making under deep uncertainty within the context of a complex harvested predator-prey fishery. 

The first post in this training series, [Fisheries Training 0: Exploring Predator-Prey Dynamics, can be found here](https://waterprogramming.wordpress.com/2022/07/11/__trashed-3/). 

The GitHub repository, containing all of the source code used throughout this series, is available [here](https://github.com/TrevorJA/harvested_predator_prey_system_tutorial_series). 

## Harvested predator-prey model (Trevor)

Re-vist the ODEs here.



## Harvesting policy - Radial Basis Functions




The function ```harvest_streategy()``` is contained within the ```fish_game``` script, which can be accessed [here in the repository](https://github.com/TrevorJA/harvested_predator_prey_system_tutorial_series/blob/main/Part_1_MOEA_Diagnostics/fish_game.py).

The ```harvest_strategy()``` function uses the RBF parameter values (i.e., \\(c_i, b_i,\\) and w_i)

returns ...

```python
def harvest_strategy(population, RBF_params, nRBFs = 2):
    ...
    harvest_effort = ...
    ...
    return harvest_effort 
```

At every timestep 

## Harvest strategy objectives (Trevor)


#### Objective #1: 


#### Objective #2:


#### Objective #3:


#### Objective #4:



## Building a model of the harvested system

Here, I provide an overview of the ```fish_game()``` model which combines many of the preceeding topics. The goal for this model is to take a set of RBF parameters, which define the harvesting strategy, simulate the policy for some length of time, and then return the objective score. 

Later, this model is what will be used with a Multi-Objective Evolutionary Algorithm, which will evaluate thousands of policies (RBF parameter combinations) and attempt to find those policies with the best objective performance. 

The model process is...


```python
def fish_game(RBF_params):

    return objectives, constraint
```

In [ ]:
# Import common libraries
import numpy as np
import matplotlib.plot as plt


In [ ]:
# Code for the fish game here, with objective outputs

def fish_game():
    
    return


## PyBorg (Lillian)

Brief summary of pyborg.

Mention how it is paired with Platypus as well

## Installation (Lillian)

We should be very detailed here, I noticed that Antonia's pyborg post was not super detailed/clear. 

Also, the ```setup.py``` method she describes is outdated, and prompts a warning.  Instead, we can just mention the dependencies and recommend pip installs for each (i think Platypus and Six are the only non-common libraries)

### Initializing the problem

Show how to define decision variables, etc.




In [ ]:
# Code for initializing the problem using PyBorg
problem = ...
algorithm = BorgMOEA()


### Run the algorithm

First, run the algorithm with a small NFE to make sure things are working. 


In [ ]:
# Run the algorithm
algorithm.run(300)

# Plot the results
plt.plot()

The objectives scores arn't very good, but that is because the number of function evaluations is so low. In order to get a better set of solutions, we need to run the MOEA for many function evaluations.  

The next section demonstrates the change in objective performance with respect to the number of function evaluations.


### MOEA Diagnostics

Explain why to do diagnostic tests.

Reed group Alum Prof. [Joe Kasprzyk wrote a post describing three common MOEA diagnostic metrics](https://waterprogramming.wordpress.com/2013/06/25/moea-performance-metrics/): generational distance, epsilon indicator, and hypervolume. 

#### Hypervolume

The hypervolume is a measure of the multi-dimensional volume _dominated_ by the approximated pareto front. As the pareto front advances toward the "_ideal_" solution, this value approaches 1. 

The efficiency of an MOEA in optimizing a solution can be considered by measuring the hypervolume with respect to the number of function evaluations. This allows the user to understand how quickly the MOEA is converging to a good set of solutions, and how many function evaluations are needed to achieve a good set of solutions.
